In [1]:
from pyflink.table import TableDescriptor, TableEnvironment, StreamTableEnvironment, EnvironmentSettings, Schema, DataTypes, WriteMode

In [2]:
from pyflink.table import CsvTableSink

In [3]:
from pyflink.table.expressions import col as col_fk

In [4]:
from pyflink.datastream import StreamExecutionEnvironment

In [5]:
stream_env = StreamExecutionEnvironment.get_execution_environment()

In [6]:
env_settings = EnvironmentSettings.new_instance()\
                                .in_streaming_mode()\
                                .build()

In [7]:
stream_tbl_env = StreamTableEnvironment.create(stream_execution_environment=stream_env, environment_settings=env_settings)

In [8]:
# stream_tbl_env.get_config().get_configuration().set_string("parallelism.default", "1")

In [9]:
# stream_env.set_parallelism(1)

In [10]:
stream_tbl_sch_source = Schema.new_builder()\
                            .column_by_metadata('source_date', DataTypes.TIMESTAMP(),'timestamp')\
                            .column('seller_id', DataTypes.STRING())\
                            .column('product', DataTypes.STRING())\
                            .column('quantity', DataTypes.INT())\
                            .column('product_price', DataTypes.DOUBLE())\
                            .column('sales_date', DataTypes.DATE())\
                            .build()

In [11]:
stream_tbl_dis_source = TableDescriptor.for_connector('kafka')\
                                    .schema(stream_tbl_sch_source)\
                                    .option('topic','productsales')\
                                    .option('properties.group.id','source-demo-1')\
                                    .option('properties.bootstrap.servers','localhost:9092')\
                                    .option('scan.startup.mode','earliest-offset')\
                                    .format('json')\
                                    .option('json.ignore-parse-errors','true')\
                                    .build()

In [12]:
stream_tbl_env.create_table('kafka_stream_tbl_source',stream_tbl_dis_source)

In [13]:
kafka_stream_tbl_source = stream_tbl_env.scan('kafka_stream_tbl_source')

In [14]:
kafka_stream_tbl_source =  kafka_stream_tbl_source.select(col_fk('*'))\
                                                .group_by(col_fk('source_date'), col_fk('seller_id'), col_fk('product'))\
                                                .select(col_fk('source_date'),
                                                        col_fk('seller_id'),
                                                        col_fk('product'), 
                                                        col_fk('quantity').avg.alias('avg_quantity'), 
                                                        col_fk('product_price').avg.alias('avg_product_price'))

In [15]:
# kafka_stream_tbl_source =  kafka_stream_tbl_source.select(col_fk('*'))\
#                                                 .group_by(col_fk('seller_id'), col_fk('product'))\
#                                                 .select(col_fk('seller_id'),
#                                                         col_fk('product'), 
#                                                         col_fk('quantity').avg.alias('avg_quantity'), 
#                                                         col_fk('product_price').avg.alias('avg_product_price'))

In [16]:
kafka_stream_ds_source = stream_tbl_env.to_changelog_stream(kafka_stream_tbl_source)

In [17]:
kafka_stream_tbl_source = stream_tbl_env.from_data_stream(kafka_stream_ds_source)

In [18]:
csv_stream_tbl_sink = CsvTableSink(field_names=kafka_stream_tbl_source.get_schema().get_field_names(), 
                                   field_types=kafka_stream_tbl_source.get_schema().get_field_data_types(), 
                                   path='./CSV/product_sale.csv',
                                   num_files=1,
                                   write_mode=WriteMode.OVERWRITE)

In [19]:
stream_tbl_env.register_table_sink('csv_stream_tbl_sink',csv_stream_tbl_sink)

In [20]:
kafka_stream_tbl_source.execute_insert('csv_stream_tbl_sink')